In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import glob
import matplotlib
import matplotlib.pyplot as plt
import cftime
import dask
from dask_jobqueue import PBSCluster
from dask.distributed import Client
import statsmodels.api as sm

In [2]:
def get_ensemble(files,data_vars,p=True):

    def preprocess(ds):
        return ds[data_vars]

    #read in the dataset
    ds = xr.open_mfdataset(files,combine='nested',concat_dim='ens',
                           parallel=p,preprocess=preprocess)

    #fix up time dimension
    htape='h0'
    #if htape=='h0' or htape=='h1':
    #    ds['time'] = xr.cftime_range(str(2005),periods=len(ds.time),freq='MS') #fix time bug

    #specify extra variables    
    if htape=='h0':
        extras     = ['grid1d_lat','grid1d_lon']
    elif htape=='h1':
        extras     = ['pfts1d_lat','pfts1d_lon','pfts1d_wtgcell','pfts1d_itype_veg']
    
    #add in some extra variables
    ds0 = xr.open_dataset(files[0])
    for extra in extras:
        ds[extra]=ds0[extra]

    return ds

In [3]:
# Setup your PBSCluster
ncores=1
nmem='25GB'
cluster = PBSCluster(
    cores=ncores, # The number of cores you want
    memory=nmem, # Amount of memory
    processes=1, # How many processes
    queue='casper', # The type of queue to utilize (/glade/u/apps/dav/opt/usr/bin/execcasper)
    local_directory='$TMPDIR', # Use your local directory
    resource_spec='select=1:ncpus='+str(ncores)+':mem='+nmem, # Specify resources
    project='P93300641', # Input your project ID here
    walltime='03:00:00', # Amount of wall time
    interface='ib0', # Interface to use
)

# Scale up
cluster.scale(20)

# Setup your client
client = Client(cluster)

In [4]:
#fetch the paraminfo
csv = '/glade/scratch/djk2120/PPEn11/SP_bfb_test.csv' 
paramkey = pd.read_csv(csv)

#fetch the sparsegrid landarea
la_file = '/glade/scratch/djk2120/PPEn08/sparsegrid_landarea.nc'
la = xr.open_dataset(la_file).landarea  #km2

In [24]:
kdir = '/glade/scratch/oleson/'
keys  = []; params = []
files = []
for key,param in zip(paramkey.key,paramkey.param):
    thisdir = kdir+'PPEn11_CTL2010SP_'+key+'/run/'
    rfile   = glob.glob(thisdir+'*.clm2.r.*.nc')
    if len(rfile)>0:
        keys.append(key)
        params.append(param)
        h0 = glob.glob(thisdir+'*.clm2.h0.*.nc')
        files.append(h0[0])
params=np.array(params)

In [6]:
datavars = ['FPSN','EFLX_LH_TOT','FSA','TV','TSOI_10CM','SOILWATER_10CM']
ds =get_ensemble(files,datavars)

In [26]:
nens = len(ds.ens)
bfb_all = np.zeros(nens)+1
for v in datavars:

    x0  = ds[v].sel(ens=0)
    bfb_grid = (ds[v]==x0).values
    
    isnan = np.tile(np.isnan(x0),[nens,1,1])
    bfb_grid[isnan]=1                       #ignore nans

    bfb = bfb_grid.sum(axis=(1,2))==24*400            #all gridcells / all times must be BFB
    print(v,bfb.sum())
    
    bfb_all = bfb_all*bfb
bfb = bfb_all==1

FPSN 75
EFLX_LH_TOT 75
FSA 75
TV 75
TSOI_10CM 75
SOILWATER_10CM 75


In [27]:
params[bfb]

array(['default', 'grperc', 'br_mr', 'lmr_intercept_atkin',
       'FUN_fracfixers', 'fun_cn_flex_a', 'fun_cn_flex_b',
       'fun_cn_flex_c', 'kc_nonmyc', 'kn_nonmyc', 'akc_active',
       'akn_active', 'ekc_active', 'ekn_active', 'stem_leaf',
       'croot_stem', 'flivewd', 'frootcn', 'leaf_long', 'lwtop_ann',
       'ndays_off', 'ndays_on', 'tau_cwd', 'tau_l1', 'tau_l2_l3',
       'tau_s1', 'tau_s2', 'tau_s3', 'q10_mr', 'minpsi_hr', 'maxpsi_hr',
       'rf_l1s1_bgc', 'rf_l2s1_bgc', 'rf_l3s2_bgc', 'rf_s2s1_bgc',
       'rf_s2s3_bgc', 'rf_s3s1_bgc', 'cn_s3_bgc', 'decomp_depth_efolding',
       'max_altdepth_cryoturbation', 'max_altmultiplier_cryoturb',
       'cryoturb_diffusion_k', 'som_diffus', 'k_nitr_max_perday',
       'denitrif_respiration_coefficient',
       'denitrif_respiration_exponent',
       'denitrif_nitrateconc_coefficient',
       'denitrif_nitrateconc_exponent', 'r_mort', 'fsr_pft', 'fd_pft',
       'prh30', 'ignition_efficiency', 'cc_dstem', 'cc_leaf', 'cc_lstem',
 

In [28]:
params[~bfb]

array(['taulnir', 'taulvis', 'tausnir', 'tausvis', 'rholnir', 'rholvis',
       'rhosnir', 'rhosvis', 'xl', 'displar', 'dleaf', 'z0mr', 'csoilc',
       'cv', 'a_coef', 'a_exp', 'zlnd', 'zsno', 'd_max',
       'frac_sat_soil_dsl_init', 'lai_dl', 'z_dl', 'zetamaxstable',
       'wind_min', 'tkd_sand', 'tkd_clay', 'tkd_om', 'tkm_om', 'pd',
       'csol_om', 'csol_sand', 'csol_clay', 'bsw_sf', 'hksat_sf',
       'sucsat_sf', 'watsat_sf', 'baseflow_scalar',
       'maximum_leaf_wetted_fraction', 'interception_fraction',
       'aq_sp_yield_min', 'fff', 'liq_canopy_storage_scalar',
       'snow_canopy_storage_scalar', 'e_ice', 'n_baseflow', 'n_melt_coef',
       'accum_factor', 'eta0_vionnet', 'drift_gs', 'ssi', 'wimp',
       'upplim_destruct_metamorph', 'wind_snowcompact_fact', 'rho_max',
       'tau_ref', 'snowcan_unload_wind_fact', 'snowcan_unload_temp_fact',
       'snw_rds_refrz', 'scvng_fct_mlt_sf', 'ceta', 'medlynslope',
       'medlynintercept', 'fnps', 'theta_psii', 'theta_ip', 't

In [29]:
(~bfb).sum()

114

### reformat the BFB array to line up with the google spreadsheet

In [13]:
import os
data_url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQs413GtLXtHVDCqEPgAwn4BbDjoWmV7uFqOAWH4mgpxXoVfN6ijnJdhyRgLkV-n2eU-sSQush4CzYU/pub?output=csv'
cmd = 'curl -L '+data_url+' > p.csv' # need to add -L option to force redirects
os.system(cmd)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   426    0   426    0     0   2565      0 --:--:-- --:--:-- --:--:--  2581
100 66431    0 66431    0     0   177k      0 --:--:-- --:--:-- --:--:--  177k


0

In [42]:
df = pd.read_csv('p.csv')
plist = []
spvar = []

In [43]:
for p in df.name:
    if p in params:
        plist.append(p)
        if p in params[bfb]:
            spvar.append(0)
        else:
            spvar.append(1)
    else:
        plist.append('')
        spvar.append('')

/glade/work/djk2120/miniconda3/envs/ppe-py/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


In [47]:

dfout = pd.DataFrame({'param':plist,'SP_active?':spvar})
dfout.to_csv('sp_active.csv')

### create an SP ensemble paramkey

In [93]:
#need to look through the three existing paramkeys 
df1 = pd.read_csv('/glade/scratch/djk2120/PPEn11/surv.csv')
df2 = pd.read_csv('/glade/scratch/djk2120/PPEn11/bgc_bfb.csv')
df3 = pd.read_csv('/glade/scratch/djk2120/PPEn11/extras_for_sp.csv')
dfs = [df1,df2,df3]

In [94]:
#first add the nonbfb bgc variables
pvals=['default']; kvals=['OAAT0000']; mvals=['default']
for p in params[~bfb]:
    found=False
    for df in dfs:
        if p in df.param.values:
            found=True
            ix = df.param==p
            d  = df[ix]
            for k,p,m in zip(d.key,d.param,d.minmax):
                kvals.append(k)
                pvals.append(p)
                mvals.append(m)
            break
    if not found:
        print(p,' not found!')
        

In [95]:
#repeat for the extra SP variables
for p in ['flnr','dbh','leaf_mr_vcm','fnitr']:
    found=False
    for df in dfs:
        if p in df.param.values:
            found=True
            ix = df.param==p
            d  = df[ix]
            for k,p,m in zip(d.key,d.param,d.minmax):
                kvals.append(k)
                pvals.append(p)
                mvals.append(m)
            break
    if not found:
        print(p,' not found!')

In [99]:
np.argsort(kvals)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 107, 108, 109, 110, 111, 112, 113, 115, 117, 118, 119, 120,
       121, 122, 123, 124, 125, 126, 127, 128, 129, 215, 216, 217, 130,
       131, 132, 133, 134, 136, 137, 138, 140, 141, 142, 144, 145, 146,
       147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159,
       160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172,
       173, 174, 176, 177, 178, 180, 181, 182, 183, 184, 185, 18

In [100]:
#sort by key
ix = np.argsort(kvals)
k = np.array(kvals)[ix]
p = np.array(pvals)[ix]
m = np.array(mvals)[ix]

In [101]:
dfout = pd.DataFrame({'key':k,'param':p,'minmax':m})
dfout.to_csv('/glade/scratch/djk2120/PPEn11/SP_ensemble.csv',index=False)